# Coleta das bases usadas na análise

Nome: Enzo Cardoso de Almeida Santos Neto

Nome: Luis Fernando Monteiro da Costa

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np

## Retirando os tweets

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

In [4]:
#Produto escolhido:
produto = 'iPhone Xr'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 429

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

## Junção de duas bases

- já apagamos as duas bases que usamos para a base final, mas aqui está o código que usamos.

In [14]:
#ler os dois excel
excel_1 = pd.read_excel("iPhone Xr.xlsx")
excel_2 = pd.read_excel("iPhone Xr1.xlsx")

In [15]:
#retirar as repeticoes da coluna de treinamento
sem_repeticao_1 = set(excel_1.Treinamento)
sem_repeticao_2 = set(excel_2.Treinamento)

In [16]:
#transformar em uma série
series_1 = pd.Series(list(sem_repeticao_1)) #após o set, precisa enviar como uma lista, por isso o list()
series_2 = pd.Series(list(sem_repeticao_2))

In [17]:
#juntar as series com o pd.concat()
series_juntado = pd.concat([series_1,series_2])

In [18]:
#retirar possíveis duplicaçoes novamente
base_final = set(series_juntado)

#voltar para uma serie
series_final = pd.Series(list(base_final))

In [19]:
#criar o excel final
series_final.to_excel("base_iphoneXR_t.xlsx")

## Transformar o excel em um arquivo .txt

In [ ]:
base_acai = pd.read_excel("base_acai.xlsx")

In [ ]:
relevante = base_acai[base_acai.Classe == 0]
irrelevante = base_acai[base_acai.Classe == 1]

In [ ]:
series_relevante = pd.Series(list(relevante.Treinamento))
series_irrelevante = pd.Series(list(irrelevante.Treinamento))

In [ ]:
relevante_lista = []
for tweets in series_relevante:
    relevante_lista.append(tweets)

relevante_texto=', '.join(relevante_lista)

with open('relevante.txt','w', encoding="utf8") as arquivo:
    arquivo.write(relevante_texto)

In [ ]:
irrelevante_lista = []
for tweets in series_irrelevante:
    irrelevante_lista.append(tweets)
    
irrelevante_texto=', '.join(irrelevante_lista)

with open('irrelevante.txt','w', encoding="utf8") as arquivo:
    arquivo.write(irrelevante_texto)